## Imports

In [16]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import ast

## Data Read-in

In [17]:
df = pd.read_csv("GSA California as of 2_7_25 - 2_7_25 Buildings.csv")

In [18]:
df = df[pd.notnull(df['LA County'])]

In [19]:
df = df[df['Owned or Leased'] == 'Federally Owned']

In [20]:
df['Full_Address'] = df['Street Address'] + ' ' + df['City'] + ' ' + df['State'] + ' ' + df['Zip Code'].astype(str)

In [23]:
df.head(1)

,Location Code,Real Property Asset Name,Installation Name,Owned or Leased,GSA Region,Street Address,City,State,Zip Code,LA County,Latitude,Longitude,Building Rentable Square Feet,Available Square Feet,Construction Date,Congressional District,Congressional District Representative Name,Building Status,Real Property Asset Type,Full_Address
183,CA0194,SSA BUILDING - HUNTINGTON PARK,NaN,Federally Owned,9,6303 RUGBY AVE,HUNTINGTON PARK,CA,90255,"City of Huntington Park, Walnut Park",33.983438,-118.22663,22952.0,0.0,1969.0,642,Robert Garcia,Active,BUILDING,6303 RUGBY AVE HUNTINGTON PARK CA 90255


In [31]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import textwrap
import ast  # Needed for literal_eval

def tooltip_html(df, row_index, max_chars=30):
    """
    Generate HTML content for the tooltip with line breaks at word boundaries.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing the property data.
    - row_index (int): The index of the row in the DataFrame.
    - max_chars (int): Maximum number of characters per line.
    
    Returns:
    - str: HTML string for the tooltip content.
    """
    # Retrieve the values from the DataFrame
    name = df.at[row_index, 'Installation Name']
    address = df.at[row_index, 'Street Address']
    
    # Convert each field to a string or default to 'N/A' if missing
    name_str = str(name).strip() if pd.notnull(name) else 'N/A'
    address_str = str(address).strip() if pd.notnull(address) else 'N/A'
    
    # Wrap the address to ensure it fits nicely in the tooltip
    wrapped_address = '<br>'.join(textwrap.wrap(address_str, width=max_chars, break_long_words=False))
    
    # Construct the tooltip HTML content with labels for each field
    tooltip_content = f"""
    <div class="popup-content">
        <strong>Name:</strong> {name_str}<br>    
        <strong>Address:</strong> {wrapped_address}<br>
    </div>
    """
    return tooltip_content

# Assuming df is already defined and loaded with your data
# Initialize the map centered around the first geocoded point
map_center = df[['Latitude','Longitude']].iloc[0].tolist()
m = folium.Map(location=map_center, zoom_start=10)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles='https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# Add custom CSS to style the tooltips
custom_css = """
<style>
    .popup-content {
        max-width: 600px; /* Adjusts the maximum width of the tooltip */
        font-size: 12px;   /* Adjusts the font size */
        word-wrap: break-word; /* Ensures long words wrap to the next line */
    }
    .leaflet-tooltip {
        background-color: white;
        border: 1px solid gray;
        border-radius: 3px;
        padding: 5px;
    }
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

# Create and add the title to the map
title_html = '''
    <h3 align="center" style="font-size:16px"><b>Federally Owned Buildings in LA County</b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Initialize a MarkerCluster to handle multiple markers efficiently
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame to create markers
for i, row in df.iterrows():
    # Create tooltip HTML content with all fields
    tooltip_content = tooltip_html(df, i)
    
    # Create a CircleMarker for each row
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        tooltip=folium.Tooltip(tooltip_content, sticky=True),
        color='red',
        fill=True,
        fill_color='red',
        radius=9,
        weight=1,        # Thickness of the circle border
        opacity=1,       # Opacity of the circle border
        fill_opacity=1   # Opacity of the fill
    ).add_to(marker_cluster)

# Add layer control to toggle different layers
folium.LayerControl().add_to(m)

# Display the map
m


In [32]:
m.save('index.html')

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/GSA_California
